## How to use the UniRep mLSTM "babbler". This version demonstrates the 64-unit and the 1900-unit architecture. 

We recommend getting started with the 64-unit architecture as it is easier and faster to run, but has the same interface as the 1900-unit one.

Use the 64-unit or the 1900-unit model?

In [1]:
USE_FULL_1900_DIM_MODEL = False # if True use 1900 dimensional model, else use 64 dimensional one.

## Setup

In [2]:
import tensorflow as tf
import numpy as np

# Set seeds
tf.set_random_seed(42)
np.random.seed(42)

if USE_FULL_1900_DIM_MODEL:
    # Sync relevant weight files
    !aws s3 sync --no-sign-request --quiet s3://unirep-public/1900_weights/ 1900_weights/
    
    # Import the mLSTM babbler model
    from unirep import babbler1900 as babbler
    
    # Where model weights are stored.
    MODEL_WEIGHT_PATH = "./1900_weights"
    
else:
    # Sync relevant weight files
    !aws s3 sync --no-sign-request --quiet s3://unirep-public/64_weights/ 64_weights/
    
    # Import the mLSTM babbler model
    from unirep import babbler64 as babbler
    
    # Where model weights are stored.
    MODEL_WEIGHT_PATH = "./64_weights"

## Data formatting and management

Initialize UniRep, also referred to as the "babbler" in our code. You need to provide the batch size you will use and the path to the weight directory.

In [3]:
batch_size = 12
b = babbler(batch_size=batch_size, model_path=MODEL_WEIGHT_PATH)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


UniRep needs to receive data in the correct format, a (batch_size, max_seq_len) matrix with integer values, where the integers correspond to an amino acid label at that position, and the end of the sequence is padded with 0s until the max sequence length to form a non-ragged rectangular matrix. We provide a formatting function to translate a string of amino acids into a list of integers with the correct codex:

You could use your own data flow as long as you ensure that the data format is obeyed. Alternatively, you can use the data flow we've implemented for UniRep training, which happens in the tensorflow graph. It reads from a file of integer sequences, shuffles them around, collects them into groups of similar length (to minimize padding waste) and pads them to the max_length. Here's how to do that:

First, sequences need to be saved in the correct format. Suppose we have a new-line seperated file of amino acid sequences, `seqs.txt`, and we want to format them. Note that training is currently only publicly supported for amino acid sequences less than 275 amino acids as gradient updates for sequences longer than that start to get unwieldy. If you want to train on sequences longer than this, please reach out to us. 

Sequence formatting can be done as follows:

In [4]:
# Before you can train your model, 
with open("seqs.txt", "r") as source:
    with open("formatted.txt", "w") as destination:
        for i,seq in enumerate(source):
            seq = seq.strip()
            if b.is_valid_seq(seq) and len(seq) < 275: 
                formatted = ",".join(map(str,b.format_seq(seq)))
                destination.write(formatted)
                destination.write('\n')

This is what the integer format looks like

In [5]:
!head -n1 formatted.txt

24,1,2,4,13,6,6,21,18,8,13,16,16,14,17,21,16,6,21,5,13,5,16,9,13,3,4,18,7,16,2,13,6,13,6,13,5,15,8,9,13,4,21,8,21,4,18,17,11,8,8,13,4,21,14,16,14,20,14,8,21,16,8,8,21,8,19,13,16,10,11,18,15,2,19,14,5,3,1,4,10,3,5,18,18,4,7,15,1,14,6,13,19,16,10,6,2,8,17,7,18,4,5,5,13,8,19,4,8,2,15,6,16,4,18,6,13,5,8,21,16,9,2,17,6,21,4,13,17,5,18,4,6,5,13,9,17,21,13,3,4,21,6,19,9,18,9,7,3,9,16,19,17,8,15,5,4,10,4,9,13,17,4,15,9,18,4,17,2,3,9,16,6,5,13,7,16,10,21,15,5,3,19,10,10,9,8,14,17,13,5,13,14,16,21,21,14,5,9,3,19,21,7,8,10,7,16,21,7,4,5,14,9,6,4,2,5,3,1,16,21,21,6,18,16,8,15,15,13,17,8,3,13,1,5,6,21,19,4


Notice that by default format_seq does not include the stop symbol (25) at the end of the sequence. This is the correct behavior if you are trying to train a top model, but not if you are training UniRep representations.

Now we can use a custom function to bucket, batch and pad sequences from `formatted.txt` (which has the correct integer codex after calling `babbler.format_seq()`). The bucketing occurs in the graph. 

What is bucketing? Specify a lower and upper bound, and interval. All sequences less than lower or greater than upper will be batched together. The interval defines the "sides" of buckets between these bounds. Don't pick a small interval for a small dataset because the function will just repeat a sequence if there are not enough to
fill a batch. All batches are the size you passed when initializing the babbler.

This is also doing a few other things:
- Shuffling the sequences by randomly sampling from a 10000 sequence buffer
- Automatically padding the sequences with zeros so the returned batch is a perfect rectangle
- Automatically repeating the dataset

In [6]:
bucket_op = b.bucket_batch_pad("formatted.txt", interval=1000) # Large interval

Inconveniently, this does not make it easy for a value to be associated with each sequence and not lost during shuffling. You can get around this by just prepending every integer sequence with the sequence label (eg, every sequence would be saved to the file as "{brightness value}, 24, 1, 5,..." and then you could just index out the first column after calling the `bucket_op`. Please reach out if you have questions on how to do this.

Now that we have the `bucket_op`, we can simply `sess.run()` it to get a correctly formatted batch

In [7]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    batch = sess.run(bucket_op)
    
print(batch)
print(batch.shape)

[[24  1  7 ...  0  0  0]
 [24  1 10 ...  4  2 19]
 [24  1 13 ...  0  0  0]
 ...
 [24  1  2 ...  0  0  0]
 [24  1 17 ...  0  0  0]
 [24  1 15 ...  0  0  0]]
(12, 248)


You can look back and see that the batch_size we passed to __init__ is indeed 12, and the second dimension must be the longest sequence included in this batch. Now we have the data flow setup (note that as long as your batch looks like this, you don't need my flow), so we can proceed to implementing the graph. The module returns all the operations needed to feed in sequence and get out trainable representations.

## Training a top model and a top model + mLSTM.

First, obtain all of the ops needed to output a representation

In [8]:
final_hidden, x_placeholder, batch_size_placeholder, seq_length_placeholder, initial_state_placeholder = (
    b.get_rep_ops())
logits, seqloss, x_placeholder, y_placeholder, batch_size_placeholder, initial_state_placeholder = (
    b.get_babbler_ops())

`final_hidden` should be a batch_size x rep_dim matrix.

Lets say we want to train a basic feed-forward network as the top model, doing regression with MSE loss, and the Adam optimizer. We can do that by:

1.  Defining a loss function.

2.  Defining an optimizer that's only optimizing variables in the top model.

3.  Minimizing the loss inside of a TensorFlow session

In [9]:
learning_rate=.001
optimizer = tf.train.AdamOptimizer(learning_rate)
tuning_op = optimizer.minimize(seqloss)

We next need to define a function that allows us to calculate the length each sequence in the batch so that we know what index to use to obtain the right "final" hidden state

In [10]:
def nonpad_len(batch):
    nonzero = batch > 0
    lengths = np.sum(nonzero, axis=1)
    return lengths

nonpad_len(batch)

array([239, 248, 205, 111,  97, 223, 148, 239, 232, 124, 120, 230])

We are ready to train. As an illustration, let's learn to predict the number 42 just optimizing the top model.

In [14]:
num_iters = 100
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(num_iters):
        batch = sess.run(bucket_op)
        loss_, __, = sess.run([seqloss, tuning_op],
                feed_dict={
                     x_placeholder: batch,
                     y_placeholder: batch,
                     batch_size_placeholder: batch_size,
                     initial_state_placeholder:b._zero_state
                }
        )
                  
        print("Iteration {0}: {1}".format(i, loss_))

Iteration 0: 2.9193031787872314
Iteration 1: 2.671344757080078
Iteration 2: 2.4535562992095947
Iteration 3: 2.171612501144409
Iteration 4: 1.9359523057937622
Iteration 5: 1.7725015878677368
Iteration 6: 1.5833114385604858
Iteration 7: 1.4367151260375977
Iteration 8: 1.2902709245681763
Iteration 9: 1.1809102296829224
Iteration 10: 1.0812214612960815
Iteration 11: 0.9709339141845703
Iteration 12: 0.8941434025764465
Iteration 13: 0.8071114420890808
Iteration 14: 0.7381923794746399
Iteration 15: 0.6808188557624817
Iteration 16: 0.6151249408721924
Iteration 17: 0.5715473294258118
Iteration 18: 0.5192060470581055
Iteration 19: 0.47662079334259033
Iteration 20: 0.44171610474586487
Iteration 21: 0.39906564354896545
Iteration 22: 0.37184953689575195
Iteration 23: 0.3408844470977783
Iteration 24: 0.31489357352256775
Iteration 25: 0.2936560809612274
Iteration 26: 0.2700284719467163
Iteration 27: 0.253261923789978
Iteration 28: 0.23777692019939423
Iteration 29: 0.22080981731414795
Iteration 30: 0.

We can also jointly train the top model and the mLSTM. Note that if using the 1900-unit (full) model, you will need a GPU with at least 16GB RAM. To see a demonstration of joint training with fewer computational resources, please run this notebook using the 64-unit model.